In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
if "../src" not in sys.path:
    sys.path.append("../src")
    
import json
import pandas as pd
from pathlib import Path
from itertools import chain
from typing import cast, Dict
from tqdm.auto import tqdm
import numpy as np
from praatio import textgrid
from weave import Utterance, Speech, BuildUtt

In [3]:
### Read charac-boundary dictionary
word_syll_map = pd.read_csv(Path("../data/mandarin_taiwan_mfa_redelvar.syll.tsv"), sep="\t")\
                  .groupby("ipa").first().drop("word", axis=1)\
                  .to_dict(orient="index")
word_syll_map = cast(Dict[str, Dict[str, str]], word_syll_map)

In [4]:
speaker_dir = Path("../data/aligned_xianzai/s1")

for tg_path_x in tqdm(sorted(speaker_dir.glob("*.TextGrid"))):
  mfa_json_path = tg_path_x.with_suffix(".mfa.json")
  fa = textgrid.openTextgrid(str(tg_path_x), False)
  fa_words = fa.getTier("words")
  fa_phones = fa.getTier("phones")
  mfa_wlist = BuildUtt.build_words_phones(fa_words, fa_phones)  #type: ignore
  mfa_words = BuildUtt.build_characters(mfa_wlist, word_syll_map)  #type: ignore
  mfa_utt = Utterance.from_words(mfa_words)
  mfa_utt.utt_id = tg_path_x.stem
  speech_x = Speech.from_utterances([mfa_utt])
  mfa_json_path.write_text(json.dumps(speech_x.to_dict()))

  0%|          | 0/852 [00:00<?, ?it/s]

In [5]:
!cp ../data/aligned_xianzai/s1/*.mfa.json ../data/aligned_xianzai/mfa_json

In [6]:
from weave import MfaAgent
CORPUS_NAME = "xianzai"
mfa_dir = Path(f"~/Documents/MFA/{CORPUS_NAME}").expanduser()
agent = MfaAgent(str(mfa_dir), CORPUS_NAME, 
                 speech_json_dir="../data/aligned_xianzai/mfa_json")

STDERR: gmm-copy --binary=false /home/seantyh/Documents/MFA/xianzai/alignment/final.mdl - 
LOG (gmm-copy[5.5.1068]:main():gmmbin/gmm-copy.cc:75) Written model to -



## Buliding data

In [12]:
# MFCC frame duration: 25ms, shift: 10ms
# took around five minutes on T15p
phones_data = []
feats_data = []

for speech_idx, speech_x in enumerate(tqdm(agent.speeches)):
    utt_x = speech_x.utterances[0]
    word_x = utt_x.words[0]
    mfa_uttid = agent.wav2uttid[utt_x.utt_id]
    lgmmprob, feat_mat = agent.compute_gmm(mfa_uttid, return_features=True)

    start_step = int((word_x.start-utt_x.start)/0.01)
    end_step = int((word_x.end-utt_x.start)/0.01)
    end_step = min(end_step, lgmmprob.shape[0])
    
    aligned_phones = list(word_x.iter_phones())
    ali_phone_idx = 0
    cur_phone = aligned_phones[ali_phone_idx]
    for t in range(start_step, end_step):
        if (cur_phone.end-utt_x.start)/0.01 < t+0.5:
            ali_phone_idx += 1
            cur_phone = aligned_phones[ali_phone_idx]
        ml_phone = agent.phone_table[lgmmprob[t, :].argmax()]
        meta_cols = [utt_x.utt_id, t, cur_phone.label, ml_phone]
        phones_data.append([*meta_cols, *lgmmprob[t, :]])
        feats_data.append([*meta_cols, *feat_mat[t, :]])

  0%|          | 0/852 [00:00<?, ?it/s]

transform-feats /home/seantyh/Documents/MFA/xianzai/alignment/lda.mat ark:- ark:- 
LOG (transform-feats[5.5.1068]:main():featbin/transform-feats.cc:158) Overall average [pseudo-]logdet is -30.4517 over 10 frames.
LOG (transform-feats[5.5.1068]:main():featbin/transform-feats.cc:161) Applied transform to 1 utterances; 0 had errors.
gmm-compute-likes /home/seantyh/Documents/MFA/xianzai/alignment/boost.1.1.mdl ark:- ark:- 
LOG (gmm-compute-likes[5.5.1068]:main():gmmbin/gmm-compute-likes.cc:82) gmm-compute-likes: computed likelihoods for 1 utterances.
transform-feats /home/seantyh/Documents/MFA/xianzai/alignment/lda.mat ark:- ark:- 
LOG (transform-feats[5.5.1068]:main():featbin/transform-feats.cc:158) Overall average [pseudo-]logdet is -30.4517 over 16 frames.
LOG (transform-feats[5.5.1068]:main():featbin/transform-feats.cc:161) Applied transform to 1 utterances; 0 had errors.
gmm-compute-likes /home/seantyh/Documents/MFA/xianzai/alignment/boost.1.1.mdl ark:- ark:- 
LOG (gmm-compute-likes[5

In [16]:
import pandas as pd
phones_df = pd.DataFrame(phones_data, columns=["utt_id", "t", "phone", "ml_phone", *agent.phone_table])
feats_df = pd.DataFrame(feats_data, columns=["utt_id", "t", "phone", "ml_phone", *range(feat_mat.shape[1])])

In [17]:
phones_df.shape, feats_df.shape

((25942, 49), (25942, 44))

In [18]:
phones_df_path = "../data/xianzai_phones_loglik.csv"
feats_df_path = "../data/xianzai_feats.csv"
phones_df.to_csv(phones_df_path, index=False)
feats_df.to_csv(feats_df_path, index=False)
!sha1sum $phones_df_path $feats_df_path

3e4803b899e4ce11e57640598fa0276234c0891c  ../data/xianzai_phones_loglik.csv
e7714cc6f816f2c65ef3046f38f5bfacaf67e0d4  ../data/xianzai_feats.csv
